In [74]:
import pandas as pd
import logging

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO, filename="Revenue_by_region_log.log",filemode="w")





payments= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\payment.csv')
orders= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\orders.csv')

cafes=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\cafe.csv')
address=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\address.csv')

merged_df=payments.merge(orders,on='payment_id',how='left',indicator=True)

payment_date_converted=pd.DatetimeIndex(merged_df['payment_date']) #convert from object to datetime payment_date_converte

merged_df['Year']=payment_date_converted.year
#merged_df['Quarter']=
merged_df['Month']=payment_date_converted.month  

merged_df.sort_values(by='payment_date',ascending=True,inplace=True)
merged_df.drop_duplicates(subset='payment_id',keep='first',inplace=True)#removing duplicates with same payment_id
columns_todrop=['customer_id_x','emp_id','created_at','payment_type', #removing columns that are not needed
                'order_id','item_id','customer_id_y','delivery',
                'delivery_address_id','item_quantity',
                'customer_modifications','delivery_notes','_merge']
merged_df.drop(columns=columns_todrop,axis=1,inplace=True)

columns=['cafe_id','payment_date','Year','Month','payment_id','amount']
merged_df=merged_df[columns]
merged_df['payment_date']=pd.to_datetime(merged_df['payment_date'])
merged_df['Year']=merged_df['Year'].astype('str')
merged_df['Month']=merged_df['Month'].astype('str')
#using .loc to change all values in column 'Month' to 2 digits
merged_df['Month'] = merged_df['Month'].str.zfill(2)
merged_df['Year_Month']=merged_df['Year']+'-'+merged_df['Month']

#rearranging columns and ordering by cafe_id and Year_Month
first_column = merged_df.pop('Year_Month') 
merged_df.insert(0, 'Year_Month', first_column)
first_column = merged_df.pop('cafe_id') 
merged_df.insert(0, 'cafe_id', first_column)

#calculating monthly revenue by location
merged_df.sort_values(by=['cafe_id','Year_Month'],ascending=True,inplace=True)

cafe_locations_monthly_df=merged_df.groupby(['cafe_id','Year_Month','Year','Month']).agg(
    monthly_revenue_total=('amount','sum')
).reset_index()

cafe_locations_monthly_df=cafe_locations_monthly_df.reset_index(drop=True)

cafe_address_df=pd.merge(cafes,address,on='address_id',how='left')

revenue_df=pd.merge(cafe_locations_monthly_df,cafe_address_df,on='cafe_id',how='left')
revenue_df=revenue_df.reset_index(drop=True)

revenue_df.drop(columns=['manager_emp_id','phone_number','address_id','last_update_x','address_first_line',
                         'address_id','address_second_line','postal_code','last_update_y','city'],axis=1,inplace=True)


revenue_df=revenue_df.loc[:,['country','Year_Month','Year','monthly_revenue_total']]

revenue_by_region_df=revenue_df
revenue_by_region_df.loc[revenue_df['country']=="Argentina",'country']="LATAM"
revenue_by_region_df.loc[revenue_df['country']=="Peru",'country']="LATAM"
revenue_by_region_df.rename(columns={'country':'region'},inplace=True)
#aggregating annual revenues by region
revenue_by_region_annually_df=revenue_by_region_df.groupby(['region','Year'])['monthly_revenue_total'].sum()
revenue_by_region_annually_df=pd.DataFrame(revenue_by_region_annually_df)
revenue_by_region_annually_df=revenue_by_region_annually_df.reset_index()
revenue_by_region_annually_df.rename(columns={	'monthly_revenue_total':'annual_revenue_by_region'},inplace=True)
annual_total_df=pd.DataFrame(revenue_by_region_annually_df.groupby(['Year'])['annual_revenue_by_region'].sum())
annual_total_df=annual_total_df.reset_index()
annual_total_df.rename(columns={'annual_revenue_by_region':'total_annual_revenue'},inplace=True)
revenue_by_region_annually_df=pd.merge(revenue_by_region_annually_df,annual_total_df, on='Year',how='left')
revenue_by_region_annually_df['percentage_from_total_annual_revenue']=revenue_by_region_annually_df['annual_revenue_by_region']/revenue_by_region_annually_df['total_annual_revenue']*100
revenue_by_region_annually_df['percentage_from_total_annual_revenue']=revenue_by_region_annually_df['percentage_from_total_annual_revenue'].apply(lambda x: round(x,2))
revenue_by_region_annually_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by region\Revenue_by_region_USA_vs_LATAM.csv',index=False)

#aggregating annual revenues by region separately for each region (regions are USA and LATAM)
revenue_USA_annually_df=revenue_by_region_annually_df.loc[revenue_by_region_annually_df['region']=="USA"]
revenue_LATAM_annually_df=revenue_by_region_annually_df.loc[revenue_by_region_annually_df['region']=="LATAM"]

revenue_USA_annually_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by region\Annual_revenue_USA.csv',index=False)
revenue_LATAM_annually_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by region\Annual_revenue_LATAM.csv',index=False)
